In [7]:
import re
import os
from collections import deque
from sklearn.model_selection import train_test_split
from sklearn import svm 
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, classification_report, precision_recall_curve, average_precision_score 
#from sklearn.cross_validation import cross_val_score
import operator
from sklearn import metrics
import openpyxl
from pathlib import Path
import glob
import pickle 
import random
import numpy as np
import pandas as pd
import librosa as lr
import seaborn as sns
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
import soundfile as sf
from tensorflow.errors import *
import matplotlib.pyplot as plt
import librosa, librosa.display
import os, shlex,pandas as pd, subprocess
import warnings
import time
warnings.filterwarnings('ignore')

In [13]:
print('Preparing ASVspoof2019 Data features and labels')


print("Reading Paths")
feature='LPCC';

train_dataset_path='/scratch/projects/smiles/Multi_Features/Awais_new/'+feature+'/ASVSpoof2019/PA/ASVspoof2019_PA_train/flac/'
eval_dataset_path='/scratch/projects/smiles/Multi_Features/Awais_new/'+feature +'/ASVSpoof2019/PA/ASVspoof2019_PA_eval/flac/'
dev_dataset_path ='/scratch/projects/smiles/Multi_Features/Awais_new/'+feature +'/ASVSpoof2019/PA/ASVspoof2019_PA_dev/flac/'

train_dataset_load = os.listdir(train_dataset_path) 
train_raw_labels =  open('/scratch/projects/smiles/Multi_Features/Awais_new/ASVspoof2019.PA.cm.train.trn.txt', 'r')

dev_dataset_load = os.listdir(dev_dataset_path) 
dev_raw_labels =    open('/scratch/projects/smiles/Multi_Features/Awais_new/ASVspoof2019.PA.cm.dev.trl.txt', 'r')

eval_dataset_load = os.listdir(eval_dataset_path) 
eval_raw_labels =   open('/scratch/projects/smiles/Multi_Features/Awais_new/ASVspoof2019.PA.cm.eval.trl.txt', 'r')


train_names= []
train_labels= []

eval_names= []
eval_labels= []

dev_names= []
dev_labels= []


Preparing ASVspoof2019 Data features and labels
Reading Paths


In [15]:
train_dataset_path

'/scratch/projects/smiles/Multi_Features/Awais_new/LPCC/ASVSpoof2019/PA/ASVspoof2019_PA_train/flac/'

In [16]:
with open('/scratch/projects/smiles/Multi_Features/Awais_new/ASVspoof2019.PA.cm.train.trn.txt', 'r') as f:
    for i,line in enumerate(tqdm(f)): #train_raw_labels
        t_label = line.split(' ')
        t_name = t_label[1]
        #print(i)
        #time.sleep(2)
        train_names.append(t_name.strip())
        t_lab = t_label[4].strip()
        if str(t_lab)=='spoof':
            train_labels.append(0)
        elif str(t_lab)=='bonafide':
            train_labels.append(1)  
print("Done")


54000it [00:00, 1010058.18it/s]

Done


In [17]:
train_names

['PA_T_0000001',
 'PA_T_0000002',
 'PA_T_0000003',
 'PA_T_0000004',
 'PA_T_0000005',
 'PA_T_0000006',
 'PA_T_0000007',
 'PA_T_0000008',
 'PA_T_0000009',
 'PA_T_0000010',
 'PA_T_0000011',
 'PA_T_0000012',
 'PA_T_0000013',
 'PA_T_0000014',
 'PA_T_0000015',
 'PA_T_0000016',
 'PA_T_0000017',
 'PA_T_0000018',
 'PA_T_0000019',
 'PA_T_0000020',
 'PA_T_0000021',
 'PA_T_0000022',
 'PA_T_0000023',
 'PA_T_0000024',
 'PA_T_0000025',
 'PA_T_0000026',
 'PA_T_0000027',
 'PA_T_0000028',
 'PA_T_0000029',
 'PA_T_0000030',
 'PA_T_0000031',
 'PA_T_0000032',
 'PA_T_0000033',
 'PA_T_0000034',
 'PA_T_0000035',
 'PA_T_0000036',
 'PA_T_0000037',
 'PA_T_0000038',
 'PA_T_0000039',
 'PA_T_0000040',
 'PA_T_0000041',
 'PA_T_0000042',
 'PA_T_0000043',
 'PA_T_0000044',
 'PA_T_0000045',
 'PA_T_0000046',
 'PA_T_0000047',
 'PA_T_0000048',
 'PA_T_0000049',
 'PA_T_0000050',
 'PA_T_0000051',
 'PA_T_0000052',
 'PA_T_0000053',
 'PA_T_0000054',
 'PA_T_0000055',
 'PA_T_0000056',
 'PA_T_0000057',
 'PA_T_0000058',
 'PA_T_0000059

In [18]:
print(" Reading & Creating Features ")

train_features_space = np.zeros((len(train_names), 60), dtype='float')
print(train_features_space.shape)

for i in tqdm(range(len(train_names))): #len(names)
    t_features = pd.read_excel(train_dataset_path+train_names[i]+'.xlsx', engine='openpyxl', header=None)
    t_features = t_features.to_numpy()
    train_features_space[i]=t_features

 Reading & Creating Features 
(54000, 60)



  0%|                                                                                                                                | 0/54000 [00:00<?, ?it/s]


ValueError: could not broadcast input array from shape (0,0) into shape (60,)

In [ ]:
print(" Preparing Development Data ")  
for j in dev_raw_labels:
    d_label = j.split(' ')
    d_name = d_label[1]
    dev_names.append(d_name.strip())
    dev_lab = d_label[4].strip()
    if str(dev_lab)=='spoof':
        dev_labels.append(0)
    elif str(dev_lab)=='bonafide':
        dev_labels.append(1)  
        
dev_features_space = np.zeros((len(dev_names), 60), dtype='float')
print(dev_features_space.shape)


for j in range(len(dev_names)): #len(names)
    d_features = pd.read_excel(dev_dataset_path+dev_names[j]+'.xlsx', engine='openpyxl', header=None)
    d_features = d_features.to_numpy()
    dev_features_space[j]=d_features 
    

        

In [ ]:
print(" Preparing Evaluation Data ")  
for k in eval_raw_labels:
    e_label = k.split(' ')
    e_name = e_label[1]
    eval_names.append(e_name.strip())
    eval_lab = e_label[4].strip()
    if str(eval_lab)=='spoof':
        eval_labels.append(0)
    elif str(eval_lab)=='bonafide':
        eval_labels.append(1)
        
eval_features_space = np.zeros((len(eval_names), 60), dtype='float')
print(eval_features_space.shape)        
        
for k in range(len(eval_names)): #len(names)
    e_features = pd.read_excel(eval_dataset_path+eval_names[j]+'.xlsx', engine='openpyxl', header=None)
    e_features = e_features.to_numpy()
    eval_features_space[k]=e_features 

In [ ]:
training_labels = np.array(train_labels)
Devlopment_labels = np.array(dev_labels)
Evaluation_labels = np.array(eval_labels)
training_features = np.array(train_features_space)
Devlopment_features= np.array(dev_features_space)
Evaluation_features = np.array(eval_features_space)
 
print(" Setting Classifiers")

In [ ]:
data_type = object
svclassifier = SVC(kernel='poly')
svclassifier.fit(X_train, y_train)
g.fit(X_train, y_train)
pred = g.predict(X_test)

In [ ]:
print('classification Report')
print(classification_report(y_test, pred))
accuracy=accuracy_score(y_test, pred)
precision, recall, fscore, support = score(y_test, pred)
print('accuracy: {}'.format(accuracy))
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))
print('Done!!')